# Transfer Learning
Transfer learning is a machine learning technique where a pre-trained model, developed on a large dataset for a specific task, is reused as the starting point for a different but related task. 

The primary benefit of using transfer learning is that it allows a model to leverage knowledge gained from training on one task (usually a large dataset) and apply that knowledge to another related task. This is particularly advantageous when the new task has a limited amount of labeled data, as transfer learning can lead to faster convergence, improved generalization, and enhanced performance compared to training a model from scratch. Transfer learning is especially prevalent in deep learning applications, where pre-trained models on large datasets (e.g., ImageNet for image classification) serve as effective feature extractors for various downstream tasks.

Due to the limited amount of labeled data to train the model on, we will use transfer learning in our modelling. The models that we will attempt to implement are:
- VGG16 and VGG19
- ResNet
- Inception
- DenseNet
- MobileNet
- EfficientNet
- Xception


## The Concept of Freezing
In the context of deep learning, "freezing" refers to the practice of fixing the weights of certain layers in a neural network during the training process. When a layer is frozen, its weights are not updated or adjusted during backpropagation. This can be beneficial in various scenarios, especially when using pre-trained models or transfer learning.

When a pre-trained model is used as the starting point for a new task, freezing layers is a strategic choice that offers several advantages:

1. Retaining Pre-trained Knowledge:
> The lower layers of a deep neural network often learn generic, low-level features such as edges, textures, and basic patterns. These features are applicable across a wide range of tasks. By freezing these lower layers, you retain the pre-trained knowledge, allowing the model to leverage representations that are likely to be useful for the new task.

2. Avoiding Overfitting on Limited Data:
> When the new task has a limited amount of labeled data, there is a risk of overfitting if the model is allowed to update its parameters too much. Freezing layers prevents the model from adapting too much to the small dataset, reducing the risk of overfitting.
3. Reducing Training Time and Resource Usage:
> The training of deep neural networks, especially with large architectures, can be computationally expensive. By freezing layers, you reduce the number of parameters that need to be updated during each training iteration, leading to faster training times and reduced resource requirements.
Freezing the layers from the imported mobile net and only fitting the input layer + the layers we are adding. We do this because the model has already been pre-trained and we are only fitting it to our additions to the model which are the input layer, the pooling layer, and the densed layers.

We will be freezing the layers from the pre-trained transfer models and only fitting the layers we are adding to the transfer model to accomodate for the parameters of our data set.

## Transfer Model Implementation - VGG16 and VGG19
VGG16 and VGG19 are deep convolutional neural network architectures introduced by the Visual Geometry Group at Oxford University. VGG16 consists of 16 layers, including 13 convolutional layers and 3 fully connected layers, with small 3x3 convolutional filters. VGG19 extends this architecture to 19 layers, offering increased expressiveness but with a trade-off in terms of computational complexity. Both models are known for their simplicity and effectiveness in image classification tasks, with VGG16 achieving competitive performance on benchmarks like ImageNet.

In [5]:
input_shape = (160,160)

In [11]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Create an ImageDataGenerator with augmentation parameters
datagen = ImageDataGenerator(
    rescale=1./255,                # Rescale pixel values to the range [0, 1]
    rotation_range=15,             # Random rotation (degrees)
    width_shift_range=0.1,         # Random horizontal shift
    height_shift_range=0.1,        # Random vertical shift
    shear_range=0.1,               # Shear intensity
    zoom_range=0.1,                # Random zoom
    horizontal_flip=True,          # Random horizontal flip
    vertical_flip=False,           # No vertical flip for MRI images
    fill_mode='nearest'            # Fill mode for handling newly created pixels
)

train_generator = datagen.flow_from_directory(
    'Kaggle_DataSet1/Train',
    target_size=input_shape,
    batch_size=32,
    class_mode='categorical' 
)

val_generator = datagen.flow_from_directory(
    'Kaggle_DataSet1/Val',
    target_size=input_shape,
    batch_size=32,
    class_mode='categorical'
)

test_generator = datagen.flow_from_directory(
    'Kaggle_DataSet1/Test',
    target_size=input_shape,
    batch_size=32,
    class_mode='categorical'  
)

Found 2283 images belonging to 4 classes.
Found 330 images belonging to 4 classes.
Found 652 images belonging to 4 classes.


In [12]:
# VGG16 Implementation

import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers, models


# Load the pre-trained VGG16 model without the fully connected layers
VGG16_base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape + (3,))

# Freeze the layers of the pre-trained model
for layer in VGG16_base_model.layers:
    layer.trainable = False

# Create a new model by adding custom fully connected layers on top of the VGG16 base
VGG16_transfer_model = models.Sequential()
VGG16_transfer_model.add(VGG16_base_model)
VGG16_transfer_model.add(layers.Flatten())
VGG16_transfer_model.add(layers.Dense(512, activation='relu'))
VGG16_transfer_model.add(layers.Dropout(0.5))
VGG16_transfer_model.add(layers.Dense(4, activation='softmax'))  

# Compile the model
VGG16_transfer_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model using the data generator
VGG16_history = VGG16_transfer_model.fit(
    train_generator,
    epochs=10,
    validation_data=test_generator
)

VGG16_validation_score = VGG16_transfer_model.evaluate(val_generator)

VGG16_transfer_model.save('VGG16_transfer_model.h5')

# To reload the model use:
#   from tensorflow.keras.models import load_model
#
#   VGG16_transfer_model = load_model('VGG16_transfer_model.h5')

11/72 [===>..........................] - ETA: 5:18 - loss: 4.0045 - accuracy: 0.2983

KeyboardInterrupt: 

In [13]:
# VGG19 Implementation

import tensorflow as tf
from tensorflow.keras.applications import VGG19
from tensorflow.keras import layers, models


# Load the pre-trained VGG16 model without the fully connected layers
VGG19_base_model = VGG19(weights='imagenet', include_top=False, input_shape=input_shape + (3,))

# Freeze the layers of the pre-trained model
for layer in VGG19_base_model.layers:
    layer.trainable = False

# Create a new model by adding custom fully connected layers on top of the VGG16 base
VGG19_transfer_model = models.Sequential()
VGG19_transfer_model.add(VGG19_base_model)
VGG19_transfer_model.add(layers.Flatten())
VGG19_transfer_model.add(layers.Dense(512, activation='relu'))
VGG19_transfer_model.add(layers.Dropout(0.5))
VGG19_transfer_model.add(layers.Dense(4, activation='softmax'))  

# Compile the model
VGG19_transfer_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model using the data generator
VGG19_history = VGG19_transfer_model.fit(
    train_generator,
    epochs=10,
    validation_data=test_generator
)

VGG19_validation_score = VGG19_transfer_model.evaluate(val_generator)

VGG19_transfer_model.save('VGG19_transfer_model.h5')

# To reload the model use:
#   from tensorflow.keras.models import load_model
#
#   VGG19_transfer_model = load_model('VGG19_transfer_model.h5')

 6/11 [===============>..............] - ETA: 33s - loss: 0.8768 - accuracy: 0.6719

KeyboardInterrupt: 

## Transfer Model Implementation - ResNet
ResNet (Residual Network) is a deep convolutional neural network architecture designed to overcome the challenges of training very deep neural networks. Introduced by Microsoft Research, ResNet introduces residual connections, allowing for the direct flow of information across layers and enabling the successful training of extremely deep networks by mitigating the vanishing gradient problem.

In [14]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load the pre-trained ResNet50 model (excluding the top layers)
resnet50_base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape + (3,))

# Freeze the layers of the pre-trained model
for layer in resnet50_base_model.layers:
    layer.trainable = False

# Create a new model by adding custom classification layers on top of the ResNet50 base
resnet50_transfer_model = models.Sequential()
resnet50_transfer_model.add(resnet50_base_model)
resnet50_transfer_model.add(layers.GlobalAveragePooling2D())
resnet50_transfer_model.add(layers.Dense(512, activation='relu'))
resnet50_transfer_model.add(layers.Dropout(0.5))
resnet50_transfer_model.add(layers.Dense(4, activation='softmax'))  

# Compile the model
resnet50_transfer_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model using the data generator
history = resnet50_transfer_model.fit(
    train_generator,
    epochs=10,
    validation_data=test_generator
)

resnet50_validation_score = resnet50_transfer_model.evaluate(val_generator)

resnet50_transfer_model.save('resnet50_transfer_model.h5')

# To reload the model use:
#   from tensorflow.keras.models import load_model
#
#   resnet50_transfer_model = load_model('resnet50_transfer_model.h5')

11/11 [==============================] - 20s 2s/step - loss: 1.3135 - accuracy: 0.3545


c:\Users\gmandl\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
